In [1]:
import sqlite3
import re

conn = sqlite3.connect("data/recipe.db")
cur = conn.cursor()

def parse_recipes(path):
    with open(path, 'r') as f:
        content = f.read()

    blocks = content.strip().split("\n\n")
    recipes = []

    for block in blocks:
        lines = block.strip().split('\n')
        name = re.match(r"Recipe:\s+(.+)", lines[0]).group(1)
        time = float(re.match(r"Craft Time:\s+([0-9.]+)", lines[1]).group(1))

        ing_start = lines.index("Ingredients:") + 1
        res_start = lines.index("Results:")

        ingredients = []
        for line in lines[ing_start:res_start]:
            if line.strip():
                ing_name, ing_count = line.strip().split()
                ingredients.append((ing_name, int(ing_count)))

        results = []
        for line in lines[res_start+1:]:
            if line.strip():
                res_name, res_count = line.strip().split()
                results.append((res_name, int(res_count)))

        recipes.append({
            "name": name,
            "craft_time": time,
            "ingredients": ingredients,
            "results": results,
        })

    return recipes

def ensure_recipe_exists(name, default_time=0.0):
    cur.execute("SELECT recipe_id FROM Recipes WHERE recipe_name = ?", (name,))
    row = cur.fetchone()
    if row:
        return row[0]
    cur.execute("INSERT INTO Recipes (recipe_name, craft_time) VALUES (?, ?)", (name, default_time))
    return cur.lastrowid

def insert_to_db(recipes):
    for recipe in recipes:
        recipe_id = ensure_recipe_exists(recipe['name'], recipe['craft_time'])

        for ing_name, ing_count in recipe['ingredients']:
            ing_id = ensure_recipe_exists(ing_name)
            cur.execute("""
                INSERT INTO Recipe_Ingredients (recipe_id, ingredient_id, ingredient_count)
                VALUES (?, ?, ?)
            """, (recipe_id, ing_id, ing_count))

        for res_name, res_count in recipe['results']:
            cur.execute("""
                INSERT INTO Recipe_Results (recipe_id, result_name, result_count)
                VALUES (?, ?, ?)
            """, (recipe_id, res_name, res_count))

    conn.commit()

recipes = parse_recipes("data/filtered_recipes.txt")
insert_to_db(recipes)
conn.close()
